In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
df = pd.read_csv('data/T_T100_SEGMENT_ALL_CARRIER.csv')
area_codes_table = pd.read_csv('data/L_WORLD_AREA_CODES.csv')
carrier_classes = pd.read_csv('data/L_CARRIER_GROUP_NEW.csv')

In [5]:
# Replace World Area Codes (WAC) with descriptions from lookup table
df = df.merge(area_codes_table, left_on='ORIGIN_WAC', right_on='Code', how='left')
df['ORIGIN_WAC'] = df['Description']
df = df.drop(columns=['Code', 'Description'])
df = df.merge(area_codes_table, left_on='DEST_WAC', right_on='Code', how='left')
df['DEST_WAC'] = df['Description']
df = df.drop(columns=['Code', 'Description'])

In [6]:
# Create simplified list of carrier class descriptions
carrier_classes_modified = []
for carrier_type in carrier_classes['Description']:
    index = carrier_type.index('Carrier')
    substr = carrier_type[:index-1]
    carrier_classes_modified.append(substr)

carrier_classes['Carrier Class'] = carrier_classes_modified
carrier_classes

,Code,Description,Carrier Class
0,0,Foreign Carriers,Foreign
1,1,Large Regional Carriers (carriers with annual ...,Large Regional
2,2,National Carriers (carriers with annual revenu...,National
3,3,Major Carriers (carriers with annual revenue o...,Major
4,4,Medium Regional Carriers (carriers with annual...,Medium Regional
5,5,Small Certificated Carriers (carrier holding c...,Small Certificated
6,6,Commuter Carriers (air taxi operator which per...,Commuter
7,7,All Cargo Carriers operating under cerificates...,All Cargo
8,9,Commuter Carriers (Air Taxi providing Essentia...,Commuter


In [7]:
# Replace carrier groups with descriptions from modified lookup table
df = df.merge(carrier_classes, left_on='CARRIER_GROUP_NEW', right_on='Code', how='left')
df = df.drop(columns=['Code', 'Description', 'CARRIER_GROUP_NEW'])

In [8]:
df['Load Factor'] = df['PASSENGERS'] / df['SEATS']
df['Load Factor'] = df['Load Factor'].fillna(0)

In [9]:
df[:10]

,DEPARTURES_SCHEDULED,DEPARTURES_PERFORMED,PAYLOAD,SEATS,PASSENGERS,FREIGHT,MAIL,DISTANCE,RAMP_TO_RAMP,AIR_TIME,UNIQUE_CARRIER,UNIQUE_CARRIER_NAME,REGION,CARRIER,CARRIER_NAME,ORIGIN_CITY_MARKET_ID,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_NM,ORIGIN_COUNTRY_NAME,ORIGIN_WAC,DEST_CITY_MARKET_ID,DEST,DEST_CITY_NAME,DEST_STATE_NM,DEST_COUNTRY_NAME,DEST_WAC,AIRCRAFT_GROUP,AIRCRAFT_TYPE,AIRCRAFT_CONFIG,YEAR,QUARTER,MONTH,DISTANCE_GROUP,CLASS,DATA_SOURCE,Carrier Class,Load Factor
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0,NEW,New England Airlines Inc.,D,NEW,New England Airlines Inc.,30613,BID,"Block Island, RI",Rhode Island,United States,Rhode Island,35855,WST,"Westerly, RI",Rhode Island,United States,Rhode Island,0,79,1,2024,1,1,1,F,DU,Commuter,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0,NEW,New England Airlines Inc.,D,NEW,New England Airlines Inc.,30613,BID,"Block Island, RI",Rhode Island,United States,Rhode Island,35855,WST,"Westerly, RI",Rhode Island,United States,Rhode Island,0,79,1,2024,3,7,1,F,DU,Commuter,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0,NEW,New England Airlines Inc.,D,NEW,New England Airlines Inc.,35855,WST,"Westerly, RI",Rhode Island,United States,Rhode Island,30613,BID,"Block Island, RI",Rhode Island,United States,Rhode Island,0,79,1,2024,1,1,1,F,DU,Commuter,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0,NEW,New England Airlines Inc.,D,NEW,New England Airlines Inc.,35855,WST,"Westerly, RI",Rhode Island,United States,Rhode Island,30613,BID,"Block Island, RI",Rhode Island,United States,Rhode Island,0,79,1,2024,3,7,1,F,DU,Commuter,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26.0,0.0,0.0,9K,Cape Air,D,9K,Cape Air,33541,MVY,"Martha's Vineyard, MA",Massachusetts,United States,Massachusetts,32250,HYA,"Hyannis, MA",Massachusetts,United States,Massachusetts,1,125,1,2024,1,1,1,F,DU,Commuter,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,0.0,0.0,9K,Cape Air,D,9K,Cape Air,33541,MVY,"Martha's Vineyard, MA",Massachusetts,United States,Massachusetts,30154,ACK,"Nantucket, MA",Massachusetts,United States,Massachusetts,1,125,1,2024,1,2,1,F,DU,Commuter,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,0.0,0.0,DL,Delta Air Lines Inc.,D,DL,Delta Air Lines Inc.,31703,HPN,"White Plains, NY",New York,United States,New York,31703,JFK,"New York, NY",New York,United States,New York,6,622,1,2024,1,2,1,L,DU,Major,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49.0,0.0,0.0,DL,Delta Air Lines Inc.,D,DL,Delta Air Lines Inc.,30721,BOS,"Boston, MA",Massachusetts,United States,Massachusetts,30721,PVD,"Providence, RI",Rhode Island,United States,Rhode Island,6,622,1,2024,3,7,1,L,DU,Major,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,54.0,0.0,0.0,9K,Cape Air,D,9K,Cape Air,34992,STX,"Christiansted, VI",U.S. Virgin Islands,United States,U.S. Virgin Islands,35626,VQS,"Vieques, PR",Puerto Rico,United States,Puerto Rico,1,125,1,2024,2,4,1,F,DU,Commuter,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80.0,0.0,0.0,DL,Delta Air Lines Inc.,D,DL,Delta Air Lines Inc.,33342,MKE,"Milwaukee, WI",Wisconsin,United States,Wisconsin,30977,MDW,"Chicago, IL",Illinois,United States,Illinois,6,622,1,2024,2,4,1,L,DU,Major,0.0
